In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import gc
%matplotlib inline
# 禁用科学计数法
pd.set_option('display.float_format',lambda x : '%.2f' % x)

## 读取数据

In [3]:
item = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian//Antai_AE_round1_item_attr_20190626.csv')
train = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian//Antai_AE_round1_train_20190626.csv')
test = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian//Antai_AE_round1_test_20190626.csv')
submit = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian//Antai_AE_round1_submit_20190715.csv')

### 数据预处理
* 合并train和test文件
* 提取日期年月日等信息
* 关联商品价格、品类、店铺
* 转化每列数据类型为可存储的最小值，减少内存消耗
* 保存为hdf5格式文件，加速读取

In [4]:
df = pd.concat([train.assign(is_train=1), test.assign(is_train=0)])

df['create_order_time'] = pd.to_datetime(df['create_order_time'])
df['date'] = df['create_order_time'].dt.date
df['day'] = df['create_order_time'].dt.day
df['hour'] = df['create_order_time'].dt.hour

df = pd.merge(df, item, how='left', on='item_id')

In [5]:

dtype_dict = {'buyer_admin_id' : 'int32', 
              'item_id' : 'int32', 
              'store_id' : 'int32',
              'irank' : 'int16',
              'item_price' : 'int16',
              'cate_id' : 'int16',
              'is_train' : 'int8',
              'day' : 'int8',
              'hour' : 'int8',
             }

df = df.fillna(0).astype(dtype_dict)
memory = df.memory_usage().sum() / 1024**2 
print('After memory usage of properties dataframe is :', memory, " MB")

After memory usage of properties dataframe is : 658.8679056167603  MB


In [ ]:
for col in ['store_id', 'item_price', 'cate_id']:
    df[col] = df[col].fillna(0).astype(np.int32).replace(0, np.nan)
df.to_hdf('I:/pythonchengx_u/Tianchiantai/dianshangtuijian/train_test1.h5', '1.0')
# df.to_hdf('../data/train_test.h5', '1.0')
print("1")

In [ ]:
%%time
df = pd.read_hdf('I:/pythonchengx_u/Tianchiantai/dianshangtuijian/train_test1.h5', '1.0')

经过前处理后:
* 文件内存占用从1200M减少至600M
* 采用hdf5格式存储，读取时间从15秒减少到仅需5秒

# Overview: 数据内容

In [ ]:
df.head()

In [ ]:
# Null 空值统计
for pdf in [df, item]:
    for col in pdf.columns:
        print(col, pdf[col].isnull().sum())

In [ ]:
df.describe()

In [ ]:
item.describe()

数据内容：
* 用户、商品、店铺、品类乃至商品价格都是从1开始用整数编号
* 订单日期格式为：YYYY-mm-dd HH:mm:ss
* 源数据中都木有空值，但是由于某些商品，不在商品表，因此缺少了一些价格、品类信息。

# 数据探查

下一步，我们依次对每个文件的特征进行基础统计和可视化处理，这是对数据进一步理解的基础。

[]~(￣▽￣)~* Let's do it.

## 训练集与测试集

In [ ]:
train = df['is_train']==1
test = df['is_train']==0

In [ ]:
train_count = len(df[train])
print('训练集样本量是',train_count)
test_count = len(df[test])
print('测试集样本量是',test_count)
print('样本比例为：', train_count/test_count)

### buyer_country_id 国家编号

In [ ]:
def groupby_cnt_ratio(df, col):
    if isinstance(col, str):
        col = [col]
    key = ['is_train', 'buyer_country_id'] + col
    
    # groupby function
    cnt_stat = df.groupby(key).size().to_frame('count')
    ratio_stat = (cnt_stat / cnt_stat.groupby(['is_train', 'buyer_country_id']).sum()).rename(columns={'count':'count_ratio'})
    return pd.merge(cnt_stat, ratio_stat, on=key, how='outer').sort_values(by=['count'], ascending=False)


In [ ]:
groupby_cnt_ratio(df, [])

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='is_train', data = df, palette=['red', 'blue'], hue='buyer_country_id', order=[1, 0])
plt.xticks(np.arange(2), ('训练集', '测试集'))
plt.xlabel('数据文件')
plt.title('国家编号');

buyer_country_id 国家编号

> 本次比赛给出若干日内来自成熟国家的部分用户的行为数据，以及来自待成熟国家的A部分用户的行为数据，以及待成熟国家的B部分用户的行为数据去除每个用户的最后一条购买数据，让参赛人预测B部分用户的最后一条行为数据。

* 训练集中有2个国家数据，xx国家样本数10635642，占比83%，yy国家样本数2232867条，仅占17%
* 预测集中有yy国家的166832数据, 训练集中yy国样本数量是测试集中的13倍，如赛题目的所言，期望通过大量成熟国家来预测少量带成熟国家的用户购买行为

### buyer_admin_id 用户编号

In [ ]:
print('训练集中用户数量',len(df[train]['buyer_admin_id'].unique()))
print('测试集中用户数量',len(df[test]['buyer_admin_id'].unique()))

In [ ]:
union = list(set(df[train]['buyer_admin_id'].unique()).intersection(set(df[test]['buyer_admin_id'].unique())))
print('同时在训练集测试集出现的有6位用户，id如下：',union)

In [ ]:
df[train][df['buyer_admin_id'].isin(union)].sort_values(by=['buyer_admin_id','irank']).head(10)

In [ ]:
df[test][df['buyer_admin_id'].isin(union)].sort_values(by=['buyer_admin_id','irank']).head(3)

In [ ]:
df[(train) & (df['irank']==1) & (df['buyer_admin_id'].isin(['12858772','3106927','12368445']))]

emmm... 为啥同一个用户在训练集和测试集国家不一样了呢？但是其他信息能对上。。。，而且rank=1的结果直接给出来了。。。

id为12858772、3106927、12368445直接把结果给出来

可能是数据清洗出问题了，后面再看看怎么处理

#### 用户记录数分布

In [ ]:
admin_cnt = groupby_cnt_ratio(df, 'buyer_admin_id')
admin_cnt.groupby(['is_train','buyer_country_id']).head(3)

In [ ]:
# 用户购买记录数——最多、最少、中位数
admin_cnt.groupby(['is_train','buyer_country_id'])['count'].agg(['max','min','median'])

In [ ]:
fig, ax = plt.subplots(1, 2 ,figsize=(16,6))
ax[0].set(xlabel='用户记录数')
sns.kdeplot(admin_cnt.loc[(1, 'xx')]['count'].values, ax=ax[0]).set_title('训练集--xx国用户记录数')

ax[1].legend(labels=['训练集', '测试集'], loc="upper right")
ax[1].set(xlabel='用户记录数')
sns.kdeplot(admin_cnt[admin_cnt['count']<50].loc[(1, 'yy')]['count'].values, ax=ax[1]).set_title('yy国用户记录数')
sns.kdeplot(admin_cnt[admin_cnt['count']<50].loc[(0, 'yy')]['count'].values, ax=ax[1]);

用户记录数进行了一波简单的探查：
* 训练集中记录了*809213*个用户的数据，其中id为10828801的用户拔得头筹，有42751条购买记录，用户至少都有8条记录
* 训练集中记录了*11398*个用户的数据，其中id为2041038的用户勇冠三军，有1386条购买记录，用户至少有7条记录

Notes: 验证集中用户最少仅有7条，是因为最后一条记录被抹去

从上面数据和图表看到，用户记录数大都都分布在0~50，少量用户记录甚至超过了10000条，下一步对用户记录数分布继续探索

In [ ]:
admin_cnt.columns = ['记录数', '占比']
admin_user_cnt = groupby_cnt_ratio(admin_cnt, '记录数')
admin_user_cnt.columns = ['人数', '人数占比']
admin_user_cnt.head()

In [ ]:
# xx国——用户记录数与用户数
admin_user_cnt.loc[(1,'xx')][['人数','人数占比']].T

In [ ]:
# yy国——记录数与用户数占比
admin_user_cnt.loc[([1,0],'yy',slice(None))][['人数','人数占比']].unstack(0).drop('人数',1).head(10)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,10))
admin_plot = admin_user_cnt.reset_index()
sns.barplot(x='记录数', y='人数占比', data=admin_plot[(admin_plot['记录数']<50) & (admin_plot['buyer_country_id']=='xx')], 
            estimator=np.mean, ax=ax[0]).set_title('训练集——xx国记录数与人数占比');

sns.barplot(x='记录数', y='人数占比', hue='is_train', data=admin_plot[(admin_plot['记录数']<50) & (admin_plot['buyer_country_id']=='yy')], 
            estimator=np.mean, ax=ax[1]).set_title('yy国记录数与人数占比');

用户记录数进一步探查结论：
    * 不管是训练集还是验证集，99%的用户购买记录都在50条内，这是比较符合正常逻辑
    * TODO:对于发生大量购买行为的用户，后面再单独探查，是否有其他规律或疑似刷单现象

### item_id 商品编号

In [ ]:
print('商品表中商品数：',len(item['item_id'].unique()))
print('训练集中商品数：',len(df[train]['item_id'].unique()))
print('验证集中商品数：',len(df[test]['item_id'].unique()))
print('仅训练集有的商品数：',len(list(set(df[train]['item_id'].unique()).difference(set(df[test]['item_id'].unique())))))
print('仅验证集有的商品数：',len(list(set(df[test]['item_id'].unique()).difference(set(df[train]['item_id'].unique())))))
print('训练集验证集共同商品数：',len(list(set(df[train]['item_id'].unique()).intersection(set(df[test]['item_id'].unique())))))
print('训练集中不在商品表的商品数：',len(list(set(df[train]['item_id'].unique()).difference(set(item['item_id'].unique())))))
print('验证集中不在商品表的商品数：',len(list(set(df[test]['item_id'].unique()).difference(set(item['item_id'].unique())))))

#### 商品销量

In [ ]:
item_cnt = groupby_cnt_ratio(df, 'item_id')
item_cnt.columns=['销量', '总销量占比']
item_cnt.reset_index(inplace=True)

In [ ]:
top_item_plot = item_cnt.groupby(['is_train','buyer_country_id']).head(10)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,12))
sns.barplot(x='item_id', y='销量', data=top_item_plot[top_item_plot['buyer_country_id']=='xx'], 
            order=top_item_plot['item_id'][top_item_plot['buyer_country_id']=='xx'], ax=ax[0], estimator=np.mean).set_title('xx国-TOP热销商品')
sns.barplot(x='item_id', y='销量', hue='is_train', data=top_item_plot[top_item_plot['buyer_country_id']=='yy'], 
            order=top_item_plot['item_id'][top_item_plot['buyer_country_id']=='yy'], ax=ax[1], estimator=np.mean).set_title('yy国-TOP热销商品');

初步数据发现：
* 训练集中出售最多商品是12691565，卖了112659次。
* 训练集中出售最多商品是5595070，卖了112659次。
* 大部分商品只有1次出售记录，符合电商长尾属性
* 比较奇怪的yy国中，训练集和测试集中热销商品并不太一样

#### 整体商品销量分布

In [ ]:
item_order_cnt = groupby_cnt_ratio(item_cnt, '销量')
item_order_cnt.columns = ['商品数', '占比']

In [ ]:
item_order_cnt.groupby(['is_train','buyer_country_id']).head(5).sort_values(by=['buyer_country_id','is_train'])

In [ ]:
item_order_plot = item_order_cnt.reset_index()
item_order_plot = item_order_plot[item_order_plot['销量']<=8]

xx_item_order_plot = item_order_plot[item_order_plot['buyer_country_id']=='xx']
yy_item_order_plot = item_order_plot[item_order_plot['buyer_country_id']=='yy']
yy_item_order_plot_1 = yy_item_order_plot[yy_item_order_plot['is_train']==1]
yy_item_order_plot_0 = yy_item_order_plot[yy_item_order_plot['is_train']==0]

In [ ]:
# 商品销量饼图
def text_style_func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}%({:d})".format(pct, absolute)

def pie_param(ax, df, color_palette):
    return ax.pie(df['占比'].values, autopct=lambda pct: text_style_func(pct, df['商品数']), labels = df['销量'], 
                  explode = [0.1]+ np.zeros(len(df)-1).tolist(), pctdistance = 0.7, colors=sns.color_palette(color_palette, 8))

fig, ax = plt.subplots(1, 3, figsize=(16,12))
ax[0].set(xlabel='xx国-商品销量')
ax[0].set(ylabel='xx国-商品数量比例')
pie_param(ax[0], xx_item_order_plot, "coolwarm")
ax[1].set(xlabel='yy国-训练集商品销量')
pie_param(ax[1], yy_item_order_plot_1, "Set3")
ax[2].set(xlabel='yy国测试集集商品销量')
pie_param(ax[2], yy_item_order_plot_0, "Set3");

In [ ]:
print(xx_item_order_plot.head(10)['占比'].sum())
print(yy_item_order_plot_1.head(10)['占比'].sum())
print(yy_item_order_plot_0.head(10)['占比'].sum())

总体来看，由于训练集数据远多于测试集数据：
* 训练集商品销量大于测试集商品销量
* 长尾趋势严重，热门商品少，大量商品仅有数次销售记录，1单商品占了绝大部分(均超过50%)
* 训练集中92%的商品销量不超过10件，而在测试集中97%的商品销量不超过10件
* 此外训练集中yy国的商品销量略大于测试集

### cate_id 品类编号

In [ ]:
print('商品品类数', len(item['cate_id'].unique()))
print('训练集商品品类数', len(df[train]['cate_id'].unique()))
print('测试集商品品类数', len(df[test]['cate_id'].unique()))

#### 各个品类下商品数量

In [ ]:
cate_cnt = item.groupby(['cate_id']).size().to_frame('count').reset_index()
cate_cnt.sort_values(by=['count'], ascending=False).head(5)

In [ ]:
plt.figure(figsize=(12,4))
sns.kdeplot(data=cate_cnt[cate_cnt['count']<1000]['count']);

我们发现：
    * 579品类一花独秀有17W个商品，可能是平台主营方向
    * 大部分品类都在100个以上

### store_id 店铺编号

In [ ]:
print('商品店铺数', len(item['store_id'].unique()))
print('训练集店铺数', len(df[train]['store_id'].unique()))
print('测试集店铺数', len(df[train]['store_id'].unique()))

#### 店铺下品类数量

In [ ]:
store_cate_cnt = item.groupby(['store_id'])['cate_id'].nunique().to_frame('count').reset_index()
store_cate_cnt.sort_values(by=['count'], ascending=False).head(5)

In [ ]:
store_cnt_cate_cnt = store_cate_cnt.groupby(['count']).size().reset_index()
store_cnt_cate_cnt.columns = ['店铺品类数', '店铺数量']

In [ ]:
plt.figure(figsize=(12,4))
sns.barplot(x='店铺品类数', y='店铺数量', data=store_cnt_cate_cnt[store_cnt_cate_cnt['店铺品类数']<50], estimator=np.mean);

#### 店铺下商品数量

In [ ]:
store_item_cnt = item.groupby(['store_id'])['item_id'].nunique().to_frame('count').reset_index()
store_item_cnt.sort_values(by=['count'], ascending=False).head(5)

In [ ]:
store_cnt_item_cnt = store_item_cnt.groupby(['count']).size().reset_index()
store_cnt_item_cnt.columns = ['店铺商品数', '店铺数量']

In [ ]:
store_cnt_item_cnt.T

In [ ]:
plt.figure(figsize=(16,4))
sns.barplot(x='店铺商品数', y='店铺数量', data=store_cnt_item_cnt[store_cnt_item_cnt['店铺商品数']<80], estimator=np.mean);

#### item_price 商品价格

In [ ]:
print(item['item_price'].max(), item['item_price'].min(), item['item_price'].mean(), item['item_price'].median())

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(121)
sns.kdeplot(item['item_price'])
plt.subplot(122)
sns.kdeplot(item['item_price'][item['item_price']<1000]);

In [ ]:
price_cnt = item.groupby(['item_price']).size().to_frame('count').reset_index()
price_cnt.sort_values(by=['count'], ascending=False).head(10)

关于商品价格：商品价格是通过函数转化成了从1开始的整数，最大值为20230，最小值为1。
    * 经常对商品价格统计，大部门商品都是整百数，Top5价格200\500\100\400\300
    * TODO：整百商品探查

#### 有售商品价格

In [ ]:
print(df[train]['item_price'].max(), df[train]['item_price'].min(), df[train]['item_price'].mean(), df[train]['item_price'].median())
print(df[test]['item_price'].max(), df[test]['item_price'].min(), df[test]['item_price'].mean(), df[test]['item_price'].median())

In [ ]:
plt.figure(figsize=(12,4))
sns.kdeplot(df[train][df[train]['item_price']<1000][['item_id','item_price']].drop_duplicates()['item_price'])
sns.kdeplot(df[test][df[test]['item_price']<1000][['item_id','item_price']].drop_duplicates()['item_price']);

商品价格与销量

In [ ]:
df[train].groupby(['item_price'])['item_id'].nunique().to_frame('商品数量').head()

In [ ]:
price_cnt = groupby_cnt_ratio(df, 'item_price')
price_cnt.groupby(['is_train', 'buyer_country_id']).head(5)

似乎价格与销量并无直接关系
    * 但是价格为100、200、300、400、500整百数位居销量榜
    * xx国，17844如此高价格的商品销量这么高？

### create_order_time 订单日期

In [ ]:
print(df[train]['create_order_time'].min(), df[train]['create_order_time'].max())
print(df[test]['create_order_time'].min(), df[test]['create_order_time'].max())

In [ ]:
train_df_seven = df[train][df[train]['create_order_time']<pd.to_datetime('2018-08-01')]
train_df_eight = df[train][df[train]['create_order_time']>pd.to_datetime('2018-08-01')]
train_df_seven = df[train][df[train]['create_order_time']<pd.to_datetime('2018-08-01')]
train_df_eight = df[train][df[train]['create_order_time']>pd.to_datetime('2018-08-01')]

In [ ]:
print('7月数据量',len(df[train][df[train]['create_order_time']<pd.to_datetime('2018-08-01')]),
      '\n8月数据量',len(df[train][df[train]['create_order_time']>pd.to_datetime('2018-08-02')]))

In [ ]:
date_cnt = groupby_cnt_ratio(df, 'date')
date_cnt.columns = ['当天销量', "占比"]
date_cnt = date_cnt.reset_index()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16,10))
sns.lineplot(x='date', y='当天销量', hue='buyer_country_id', data=date_cnt[(date_cnt['is_train']==1)], 
            estimator=np.mean, ax=ax[0]).set_title('训练集——每日销量');

sns.lineplot(x='date', y='当天销量', hue='is_train', data=date_cnt[(date_cnt['buyer_country_id']=='yy')], 
            estimator=np.mean, ax=ax[1]).set_title('yy国每日销量');

很明显：
* 训练集中7月份数据远小于8月份数据
* 训练集中xx国和yy国每日销量趋势十分相似，且在27日有个波峰

In [ ]:
seven = date_cnt[date_cnt['date']<pd.to_datetime('2018-08-02')]
eight = date_cnt[date_cnt['date']>=pd.to_datetime('2018-08-02')]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20,16))
def barplot(ax, df, title):
    df['date'] = df['date'].astype(str)
    sns.barplot(y='date', x='当天销量' ,data=df, order=sorted(df['date'].unique()), ax=ax, estimator=np.mean)\
    .set_title(title)
    
barplot(ax[0][0], seven[(seven['is_train']==1) & (seven['buyer_country_id']=='xx')], 'xx国7月份销量')
barplot(ax[1][0], eight[(eight['is_train']==1) & (eight['buyer_country_id']=='xx')], 'xx国8月份销量')
barplot(ax[0][1], seven[(seven['is_train']==1) & (seven['buyer_country_id']=='yy')], '训练集-yy国7月份销量')
barplot(ax[1][1], eight[(eight['is_train']==1) & (eight['buyer_country_id']=='yy')], '训练集-yy国8月份销量')
barplot(ax[0][2], seven[(seven['is_train']==0) & (seven['buyer_country_id']=='yy')], '测试集-yy国7月份销量')
barplot(ax[1][2], eight[(eight['is_train']==0) & (eight['buyer_country_id']=='yy')], '测试集-yy国8月份销量')
plt.tight_layout()

数据放大后看：
* 训练集和测试集在8月份有相似的波动规律，27号出现波峰，当天剧增数据有待下一步探查

#### 每日uv与商品数(去重)

In [ ]:
unique = df.groupby(['is_train', 'buyer_country_id', 'date']).agg({'buyer_admin_id':'nunique','item_id':['nunique','size']})
unique.columns = ['uv','商品数(去重)', '销量']
unique = unique.reset_index()
unique = pd.melt(unique, id_vars=['is_train', 'buyer_country_id', 'date'], value_vars=['uv', '商品数(去重)', '销量'])
unique['date'] = unique['date'].astype(str)
unique = unique[unique['date']>='2018-08-02']

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16,8), sharex=True)
sns.lineplot(x='date', y='value', hue='variable', data=unique[(unique['is_train']==1) & (unique['buyer_country_id']=='xx')], 
             estimator=np.mean, ax=ax[0]).set_title('xx国每日销售数据');

sns.lineplot(x='date', y='value', hue='variable', data=unique[(unique['is_train']==0) & (unique['buyer_country_id']=='yy')], 
            estimator=np.mean, ax=ax[1]).set_title('训练集-yy国每日销量');

sns.lineplot(x='date', y='value', hue='variable', data=unique[(unique['is_train']==1) & (unique['buyer_country_id']=='yy')], 
            estimator=np.mean, ax=ax[2]).set_title('测试集-yy国每日销量')
plt.xticks(rotation=90);

对每日的uv、商品数和销量作图发现：
* 三者基本上呈正相关，xx国的商品单品销量更高

# BASELINE
选取用户近30次购买记录作为预测值，越近购买的商品放在越靠前的列，不够30次购买记录的用热销商品5595070填充

In [7]:
test = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian/Antai_AE_round1_test_20190626.csv')
tmp = test[test['irank']<=31].sort_values(by=['buyer_country_id', 'buyer_admin_id', 'irank'])[['buyer_admin_id','item_id','irank']]
sub = tmp.set_index(['buyer_admin_id', 'irank']).unstack(-1)
sub.fillna(5595070).astype(int).reset_index().to_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian/sub.csv', index=False, header=None)

In [8]:
# 最终提交文件格式
sub = pd.read_csv('H:/pythonchengx_u/Tianchiantai/dianshangtuijian/sub.csv', header = None)
sub.head()


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,152,8410857,7937154,8472223,4016066,9891513,8064216,8351840,5595070,5595070,...,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070
1,282,11721802,7665423,7665423,10808393,11310708,623582,6547607,2605373,688799,...,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070
2,321,1461800,7379845,9243286,7379845,627849,5000759,11774753,10932288,4813286,...,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070
3,809,2347616,5707010,6339286,5492003,1207574,5707010,5492003,1207574,2262443,...,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070
4,870,11382694,5999244,6611583,7412272,4343647,5546383,3432696,9589237,6163411,...,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070,5595070
